In [ ]:
param_date = '20240309'

# 1. Part_Number修正

In [ ]:
import os

In [ ]:
list_dir = list(os.walk('./' + param_date))[0][1]
if '.DS_Store' in list_dir:
    list_dir.remove('.DS_Store')
if '0.null.txt' in list_dir:
    list_dir.remove('0.null.txt')
    
list_dir.sort()
    
len(list_dir)

In [ ]:
import re

In [ ]:
for param_dir in list_dir:
    list_file = list(os.walk('./' + param_date + '/' + param_dir))[0][2]
    if '.DS_Store' in list_file:
        list_file.remove('.DS_Store')
    if '0.null.txt' in list_file:
        list_file.remove('0.null.txt')
        
    for file in list_file:
        if file.startswith('4.part_number-'):
            os.rename('./' + param_date + '/' + param_dir + '/' + file,
                      './' + param_date + '/' + param_dir + '/' + re.sub('-[0-9]{8}_[0-9]{6}\.xlsx$', '.xlsx', file))

print('Done ~')

# 2. 数量统计

In [ ]:
import pandas as pd

df = pd.DataFrame()

In [ ]:
import os

In [ ]:
list_dir = list(os.walk('./' + param_date))[0][1]
if '.DS_Store' in list_dir:
    list_dir.remove('.DS_Store')
if '0.null.txt' in list_dir:
    list_dir.remove('0.null.txt')
    
list_dir.sort()
    
len(list_dir)

In [ ]:
from tqdm import tqdm

In [ ]:
for param_dir in tqdm(list_dir, desc='Progress', ncols=77):
    list_file = list(os.walk('./' + param_date + '/' + param_dir))[0][2]
    if '.DS_Store' in list_file:
        list_file.remove('.DS_Store')
    if '0.null.txt' in list_file:
        list_file.remove('0.null.txt')
        
    for file in list_file:
        df_temp = pd.read_excel('./' + param_date + '/' + param_dir + '/' + file,
                                header=0,
                                dtype=str).fillna('')
        
        if len(df_temp) == 0:
            continue
        
        df_temp = df_temp.groupby([df_temp['Manufacturer_No.'], df_temp['Type_Code']], as_index=False).count()
        df_temp['JOIN_MNTC'] = df_temp.loc[0, 'Manufacturer_No.'] + ';' + df_temp.loc[0, 'Type_Code']
        df_temp['Amount'] = df_temp['Part_No.']
        df_temp = df_temp[['JOIN_MNTC', 'Amount']]
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

In [ ]:
df_type = pd.read_excel('./3.type.xlsx',
                        header=0,
                        dtype=str).fillna('')

df_type

In [ ]:
df_type['JOIN_MNTC'] = [manufacturer_no + ';' + type_code for manufacturer_no, type_code in zip(df_type['Manufacturer_No.'].tolist(), df_type['Type_Code'].tolist())]
                                                                                                
df_type

In [ ]:
df_type = pd.merge(left=df_type,
                   right=df,
                   how='left',
                   left_on='JOIN_MNTC',
                   right_on='JOIN_MNTC').fillna('')

df_type

In [ ]:
list_column = df_type.columns.tolist()
list_column.remove('JOIN_MNTC')
df_type = df_type[list_column]

df_type.to_excel('./result.xlsx', index=False)

df_type